# Performance of state space sample of all the possible TensorFlow==2.4.0 on ubi8 python3.8

The purpose of this notebook is to have a reusable notebook that can be customized depending on the performance analysis that can be performed based on [Amun Service](https://github.com/thoth-station/amun-api) and using [Performance Indicators](https://github.com/thoth-station/performance).

## Amun Inspections inputs

The description of the dataset can be found [here](https://github.com/thoth-station/dependency-monkey-zoo/tree/master/tensorflow/inspection-2021-02-09).

**Software stacks and native dependencies**

  * `PyPI TensorFlow` - `tensorflow==2.4.0` available on PyPI index

**OS images**

  * `rhel-8` 

**Python Interpreters**

  * `3.8` 


**Hardware**


### Performance indicators
Performance Indicators (PI) used for performance analysis:

  * [matrix multiplication](https://github.com/thoth-station/performance/blob/master/tensorflow/matmul.py)

Each performance indicator was run `1` per inspection run (`batch size == 1`), performance indicators reported median of inspections to be further compared.

## Dataset content

Inspection specification, build logs, job logs, hardware information of the node where the performance indicator was run and the actual inspection job result.

No buildtime/runtime errors spotted with the tested stack.


# Analysis

Results of performance are shown in terms of Elapsed time [ms].

The analysis performed in this notebook are defined as:

- Randomly sampled state space of all the possible TensorFlow==2.4.0 software stacks for UBI 8 Python 3.8 to identify errors and new possible recommendations.


## Assign environment variables and import libraries

In [ ]:
%env THOTH_CEPH_KEY_ID=LLEzCoxu7pvjzO4inoL8
%env THOTH_CEPH_SECRET_KEY=1HnDVoIS2jt3h3xEpgeQlCX5+FeOUH0wOrvWVvZP
%env THOTH_CEPH_BUCKET_PREFIX=thoth
%env THOTH_S3_ENDPOINT_URL=https://s3-openshift-storage.apps.smaug.na.operate-first.cloud
%env THOTH_CEPH_BUCKET=opf-datacatalog
%env THOTH_DEPLOYMENT_NAME=datasets

In [ ]:
from thoth.report_processing.components.inspection import AmunInspections
from thoth.report_processing.components.inspection import AmunInspectionsSummary
from thoth.report_processing.components.inspection import AmunInspectionsStatistics

inspection = AmunInspections()
inspection_runs_summary = AmunInspectionsSummary()
inspection_statistics = AmunInspectionsStatistics()

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1500)
pd.options.plotting.backend = "matplotlib"  # Convert to matplotlib

In [ ]:
## Download inspections ids

import requests

url = 'https://raw.githubusercontent.com/thoth-station/dependency-monkey-zoo/master/tensorflow/inspection-2021-02-09/scheduled.txt'
r = requests.get(url)
inspection_ids_list = r.content.decode().split("\n")
print(len(inspection_ids_list))

## Retrieve and process data

In [ ]:
inspection_runs = inspection.aggregate_thoth_inspections_results(
    inspection_ids_list=inspection_ids_list,
)

In [ ]:
processed_inspection_runs, failed_inspection_runs = inspection.process_inspection_runs(
    inspection_runs,
)

In [ ]:
inspections_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=processed_inspection_runs,
    include_statistics=True
)

In [ ]:
inspections_df.head()

# Inspections summary report

In [ ]:
report_results, _ = inspection_runs_summary.produce_summary_report(inspections_df=inspections_df)

## Hardware

In [ ]:
report_results["hardware"]['platform'].head()

In [ ]:
report_results["hardware"]['processor']

In [ ]:
report_results["hardware"]['flags']

In [ ]:
report_results["hardware"]['ncpus']

In [ ]:
report_results["hardware"]['info']

## Operating System

In [ ]:
report_results["base_image"]['base_image']

In [ ]:
report_results["base_image"]['number_cpus_run']

## Performance Indicator

In [ ]:
report_results["pi"]['pi']

## Software Stack

In [ ]:
report_results["software_stack"]['requirements_locked'].head()

In [ ]:
python_packages_dataframe, python_packages_versions, versions_total_dataframe = inspection.create_python_package_df(inspections_df=inspections_df)
python_packages_dataframe.head()

# Create final dataframe

In [ ]:
final_dataframe = inspection.create_final_dataframe(
    inspections_df=inspections_df,
    include_statistics=True
)
final_dataframe.head()

# Plots

## Performance plots

In [ ]:
import os

from typing import List

import plotly.graph_objects as go

from typing import Optional, Dict
from thoth.python import Source
from plotly.subplots import make_subplots

from kaleido.scopes.plotly import PlotlyScope
scope = PlotlyScope()

python_package_index= "https://pypi.org/simple"
source = Source(python_package_index)

if not os.path.exists("images"):
    os.mkdir("images")

_PERFORMANCE_QUANTITY = ["elapsed_time", "rate"]

_PERFORMANCE_QUANTITY_MAP = {"elapsed_time": "Elapsed Time [ms]", "rate": "Rate [GFLOPS]"}


class AmunInspectionsVisualization:
    """Class of methods used to create statistics from Amun Inspections Runs."""

    @staticmethod
    def create_inspection_3d_plot(
        plot_df: pd.DataFrame,
        varying_package: str,
        title_plot: str,
        quantity: Optional[str] = 'elapsed_time',
        image_name: Optional[str] = None,
    ):
        """Create inspection performance parameters plot in 3D.

        :param plot_df: df for plots provided by `create_final_dataframe` or its subset.
        :param 
        """
        if quantity not in _PERFORMANCE_QUANTITY:
            _LOGGER.info(f"Only {_PERFORMANCE_QUANTITY} are accepted as quantity")
            return

        x_vector = [x[0] for x in plot_df[["solver_string"]].values]

        integer_y_encoded = [y[0] for y in plot_df[["sws_hash_id_encoded"]].values]

        z_vector = [z[0] for z in plot_df[[quantity]].values]
        trace1 = go.Scatter3d(
            x=x_vector,
            y=integer_y_encoded,
            z=z_vector,
            mode="markers",
            hovertext=[yc[0] for yc in plot_df[["sws_string"]].values],
            hoverinfo="text",
            marker=dict(
                size=8,
                color=z_vector,  # set color to an array/list of desired values
                colorscale="Viridis",  # choose a colorscale
                opacity=0.8,
                showscale=True,
            ),
        )

        data = [trace1]

        annotations = []
        c = 0

        for (x, y, z) in zip(x_vector, integer_y_encoded, z_vector):
            annotations.append(
                dict(
                    showarrow=False,
                    x=x,
                    y=y,
                    z=z,
                    text="".join(plot_df[varying_package].values[c]),
                    xanchor="left",
                    xshift=15,
                    opacity=0.7,
                )
            )
            c += 1

        margin = {"l": 200, "r": 100, "b": 100, "t": 100}

        layout = go.Layout(
            title=title_plot,
#             margin=margin,
            scene=dict(
                xaxis=dict(
                    title="Runtime Environment",
                    backgroundcolor="rgb(200, 200, 230)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white",
                    ),
                yaxis=dict(title="Software Stack ID",
                    backgroundcolor="rgb(230, 200,230)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white",
                    ),
                zaxis=dict(title=_PERFORMANCE_QUANTITY_MAP[quantity],
                    backgroundcolor="rgb(230, 230,200)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white"),
#                 annotations=annotations
            ),
            showlegend=True,
            legend=dict(orientation="h"),
        )
        fig = go.Figure(data=data, layout=layout)

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig
    
    
    @staticmethod
    def create_inspection_2d_plot(
        plot_df: pd.DataFrame,
        varying_package: str,
        title_plot: str,
        quantity: Optional[str] = 'elapsed_time',
        have_annotations: bool = False,
        image_name: Optional[str] = None,
    ):
        """Create inspection performance parameters plot in 2D.

        :param plot_df: df for plots provided by `create_final_dataframe` or its subset.
        """
        integer_y_encoded = [y[0] for y in plot_df[["sws_hash_id_encoded"]].values]

        data = []
        annotations = []
        colour_counter = 0
        distance: float = 0
        name_component = varying_package

        subset_df = plot_df[plot_df["pi_component"] == varying_package]
        z_vector = [z[0] for z in subset_df[[quantity]].values]

        trace = go.Scatter(
            x=integer_y_encoded,
            y=z_vector,
            mode="markers",
            hovertext=[y[0] for y in subset_df[["sws_string"]].values],
            hoverinfo="text",
            marker=dict(
                size=12,
                color=z_vector,  # set color to an array/list of desired values
#                 colorscale=color_scales[colour_counter],  # choose a colorscale
                opacity=0.8,
                showscale=True,
                colorbar={"x": 1 + distance},
            ),
            name=f"solver=={plot_df['solver_string'].unique()[0]}",
        )

        data.append(trace)
        colour_counter += 1
        distance += 0.2

        layout = go.Layout(
            title=title_plot,
            xaxis=dict(title="Software Stack ID integer encoded"),
            yaxis=dict(title=_PERFORMANCE_QUANTITY_MAP[quantity]),
            showlegend=True,
            legend=dict(orientation="h", y=-0.3, yanchor="top"),
        )
        fig = go.Figure(data=data, layout=layout)

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig
    
    @staticmethod
    def create_inspection_2d_plot_time(
        plot_df: pd.DataFrame,
        varying_package: str,
        title_plot: str,
        quantity: Optional[str] = 'elapsed_time',
        have_annotations: bool = False,
        image_name: Optional[str] = None,
    ):
        """Create inspection performance parameters plot in 2D.

        :param plot_df: df for plots provided by `create_final_dataframe` or its subset.
        """
        integer_y_encoded = [pd.to_datetime(y[0]) for y in plot_df[["start_datetime"]].values]
        data = []
        annotations = []
        colour_counter = 0
        distance: float = 0
        name_component = varying_package

        subset_df = plot_df[plot_df["pi_component"] == varying_package]
        z_vector = [z[0] for z in subset_df[[quantity]].values]

        trace = go.Scatter(
            x=integer_y_encoded,
            y=z_vector,
            mode="markers",
            hovertext=[y[0] for y in subset_df[["start_datetime"]].values],
            hoverinfo="text",
            marker=dict(
                size=12,
                color=z_vector,  # set color to an array/list of desired values
#                 colorscale=color_scales[colour_counter],  # choose a colorscale
                opacity=0.8,
                showscale=True,
                colorbar={"x": 1 + distance},
            ),
            name=f"solver=={plot_df['solver_string'].unique()[0]}",
        )

        data.append(trace)
        colour_counter += 1
        distance += 0.2

        layout = go.Layout(
            title=title_plot,
            xaxis=dict(title="Start time (datetime)"),
            yaxis=dict(title=_PERFORMANCE_QUANTITY_MAP[quantity]),
            showlegend=True,
            legend=dict(orientation="h", y=-0.3, yanchor="top"),
        )
        fig = go.Figure(data=data, layout=layout)

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig
    
    def create_packages_versions_2d_plot(
        plot_df: Dict[str, pd.DataFrame],
        title_plot: str,
        package_names: List[str],
        versions_total_df = pd.DataFrame,
        image_name: Optional[str] = None,
    ):
        """Create packages versions clusters plot in 2D.

        :param final_inspections_df: df for plots provided by `create_final_dataframe` or its subset.
        """        
        fig = make_subplots(rows=len(package_names), cols=1)
        
        ref_n = 1
        for package_name in package_names:
            
            package_versions = []

            try:
                versions = source.get_sorted_package_versions(package_name)

            except Exception as exc:
                print(str(exc))
            
            package_versions = sorted(versions_total_df[package_name].unique())

            if versions:
                
                actual_versions = [vv for vv in versions if str(vv) in package_versions]
                
                package_versions = actual_versions

            for i_ in package_versions:

                x_ = []
                y_ = []
                for name, df_ in plot_df.items():
                    y_unique = df_[package_name].unique() # packages

                    subset_df = df_[df_[package_name].str.contains(str(i_))]

                    x_cluster = [int(xx) for xx in subset_df[["sws_hash_id_encoded"]].values]  # SW ID
                    y_cluster = [name for n in subset_df[package_name].values]

                    x_ += x_cluster
                    y_ += y_cluster

                trace = go.Scatter(
                    x=x_,
                    y=y_,
                    mode="markers",
                    name=f"{package_name}-v{i_}",
                )
                
                fig.add_trace(
                    trace,
                    row=ref_n, col=1
                )
            
            layout = go.Layout(
                title=title_plot,
                xaxis=dict(title="Software Stack ID integer encoded"),
                yaxis=dict(title="clusters"),
                showlegend=True,
            )
        
            fig.update_layout(layout)
            
            ref_n += 1

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig

In [ ]:
# Images are stored under /images
title_plot = "TF==2.4.0 performance (3D plot)"
fig = AmunInspectionsVisualization.create_inspection_3d_plot(plot_df=final_dataframe, quantity="elapsed_time", varying_package="tensorflow", title_plot=title_plot)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
# Images are stored under /images
title_plot = "TF==2.4.0 performance (2D plot)"
fig = AmunInspectionsVisualization.create_inspection_2d_plot(plot_df=final_dataframe, varying_package="tensorflow", title_plot=title_plot)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
# Images are stored under /images
title_plot = "TF==2.4.0 performance (2D plot) in time"
fig = AmunInspectionsVisualization.create_inspection_2d_plot_time(plot_df=final_dataframe, varying_package="tensorflow", title_plot=title_plot)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

Most performant stack

In [ ]:
most_performant_stack = final_dataframe[final_dataframe['elapsed_time'] == final_dataframe.elapsed_time.min()]
packages = [p for p in most_performant_stack['packages_list'].values]

# check packages in that stack
for pkg in packages[0]:
    print(pkg)

Cluster the software stack to identify commonalities in the patterns identified in the plot

In [ ]:
clusters = {
    "cluster 1": final_dataframe[final_dataframe["elapsed_time"] < 425],
    "cluster 2": final_dataframe[(final_dataframe["elapsed_time"] >= 425) & (final_dataframe["elapsed_time"] < 475)],
    "cluster 3": final_dataframe[(final_dataframe["elapsed_time"] >= 475) & (final_dataframe["elapsed_time"] < 525)],
    "cluster 4": final_dataframe[(final_dataframe["elapsed_time"] >= 525) & (final_dataframe["elapsed_time"] < 575)],
    "cluster 5": final_dataframe[(final_dataframe["elapsed_time"] >= 575)],
}

cluster_processing = {}

for name, df_ in clusters.items(): 
    if name not in cluster_processing.keys():
        cluster_processing[name] = {}

    for package in sorted([k for k in python_packages_versions.keys()]):
        n_cluster = 0
        versions = final_dataframe[package].unique()
        
        y_ = df_[package].values

        for y_u in versions:
            total_number = df_[df_[package] == y_u].shape[0]
            cluster_processing[name][y_u] = total_number

clusters_df = pd.DataFrame(cluster_processing).transpose()
clusters_df

In [ ]:
package_names = ['numpy']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['cachetools']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['certifi']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['pyasn1']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['pyasn1-modules']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['requests-oauthlib']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['rsa']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['google-auth-oauthlib']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['idna']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['markdown']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

In [ ]:
package_names = ['oauthlib']
title_plot = f"Inspection analysis 20210209 software stacks clusters for packages {package_names}"
fig = AmunInspectionsVisualization.create_packages_versions_2d_plot(
    plot_df=clusters,
    package_names=package_names,
    versions_total_df=versions_total_dataframe,
    title_plot=title_plot
)
from IPython.display import Image
Image(filename=f'./images/{title_plot}.png') 

# Analyze inspection builds errors

## Failed inspection builds

In [ ]:
failed_builds_inspection_ids = set(inspection_ids_list) - set([r for r in inspections_df['inspection_document_id'].values])

In [ ]:
total_failed_builds_inspection_ids = len(failed_builds_inspection_ids)
print(f"Total number of inspection builds is {total_failed_builds_inspection_ids}")

### Retrieve inspection builds failed logs

In [ ]:
from thoth.storages import InspectionStore
store_class_type = InspectionStore

from pathlib import Path
current_path = Path.cwd()

inspections_build_files = inspection.aggregate_thoth_inspections_results(
    is_local=True,
    store_files=["build_logs"],
    repo_path=current_path.joinpath("inspectionsAnalysis20210209")
)

In [ ]:
len(inspections_build_files)

In [ ]:
failed_inspections_build_sentences = {}
for inspection_id, failed_build_logs in inspections_build_files.items():
    
    sentences = str(failed_build_logs['build_logs']).split("\n")
    if len(sentences) > 5:
        
        if sentences[-2] != 'Push successful':
            failed_inspections_build_sentences[inspection_id] = {"build_logs": sentences[-2:-1]}

print("failed inspection build logs", len(failed_inspections_build_sentences))
failed_build_logs_df = pd.DataFrame(failed_inspections_build_sentences)

for failed_inspection_build_log_sentence in ["".join(looog) for looog in failed_build_logs_df.transpose()['build_logs'].values]:
    print()
    print(failed_inspection_build_log_sentence)

In [ ]:
proces1 = ["".join(looog) for looog in failed_build_logs_df.transpose()['build_logs'].values]
timeout_error = 0
push_error = 0
other_error = 0

for p in proces1:
    
    if 'Error: received unexpected terminate signal' in p:
        timeout_error += 1
    elif 'error: build error: Failed to push image' in str(p):
        push_error += 1
    else:
        print("Other error \n", p)
        other_error += 1
        
print(f"Number of timeout error is {timeout_error}")
print(f"Number of push error is {push_error}")
print(f"Number of other error is {other_error}")

## Failed inspection runs

In [ ]:
failed_inspections_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=failed_inspection_runs,
)

In [ ]:
failed_inspections_df

## Conclusions

After initial analysis of this dataset, `801` inspections have been identified. These inspections have been created for different stack combinations of TensorFlow 2.4.0 on ubi8 with python38.

The following conclusions have been found at the moment:

**inspection builds failed**

- 51 inspections failed during builds identified.

After further analysis of this failed builds, the following causes have been identified:

** CASE 1: 27 are due to push error with the following logs:

```
error: build error: Failed to push image: error copying layers and metadata from "containers-storage:[overlay@/var/lib/containers/storage+/var/run/containers/storage:overlay.imagestore=/var/lib/shared]image-registry.openshift-image-registry.svc:5000/thoth-amun-inspection-stage/inspection-tf-dm-tf24-ID:latest" to "docker://image-registry.openshift-image-registry.svc:5000/thoth-amun-inspection-stage/inspection-tf-dm-tf24-ID:latest": Error writing manifest: Error uploading manifest latest to image-registry.openshift-image-registry.svc:5000/thoth-amun-inspection-stage/inspection-tf-dm-tf24-ID: manifest blob unknown: blob unknown to registry
```

** CASE 2: 19 appear to have been caused by some external source (maybe timeout?):

```
Error: received unexpected terminate signal
```

** CASE 3: 5 cases can be associated to CASE 2 after checking similar errors.


**inspection with results**

Analyzing the other 729 inspections:

** 0 inspections runs failed identified. No differences worth mentioning.

It appears this dataset does not contain failures that we can investigate to provide a new recommendation to users.

If no further investigations are requested, I would proceed cleaning the notebook, prepare the dataset and publish everything in this repo.


**inspection with results: performance**

New plots to check specific versions available in different clusters have been added for a deeper understanding on the software stacks differences identified in the three main clusters.